In [5]:
# Load packages without warnings and messages
suppressPackageStartupMessages({
  library(dplyr)
  library(tidyr)
  library(ggplot2)
  library(patchwork)
  library(cowplot)
  library(visdat)
  library(here)
  library(lubridate)
  library(readr)
  library(naniar)
  library(purrr)
})

In [6]:
# load drivers data
drivers <- readRDS(here::here("data/CH-Oe2_2004-2023_final_ready_for_CNmodel_run_19082025.rds"))
colnames(drivers$forcing[[1]])
str(drivers$forcing[[1]])

[1] "date"   "temp"   "vpd"    "ppfd"   "netrad" "patm"   "snow"   "rain"  
 [9] "tmin"   "tmax"   "vwind"  "fapar"  "lai"    "co2"    "ccov"   "gpp"   
[17] "le"     "nee"    "nee_qc" "dno3"   "dnh4"   "fharv"  "cseed"  "nseed"

tibble [7,300 x 24] (S3: tbl_df/tbl/data.frame)
 $ date  : Date[1:7300], format: "2004-01-01" "2004-01-02" ...
 $ temp  : num [1:7300] -0.227 -0.499 -2.956 -4.209 -1.192 ...
 $ vpd   : num [1:7300] 5.3 1 10.6 58.5 25.8 14.4 20.6 5.5 43.4 90.1 ...
 $ ppfd  : num [1:7300] 6.59e-05 4.37e-05 4.26e-05 6.59e-05 5.37e-05 ...
 $ netrad: num [1:7300] 30.3 34.7 16.3 28.2 36.5 ...
 $ patm  : num [1:7300] 95825 95455 96232 96367 96710 ...
 $ snow  : num [1:7300] 0 0 0 0 0 0 0 0 0 0 ...
 $ rain  : num [1:7300] 1.85e-05 5.79e-06 0.00 0.00 0.00 ...
 $ tmin  : num [1:7300] -1.23 -2.06 -5.82 -10.45 -8.1 ...
 $ tmax  : num [1:7300] -0.01 -0.05 -2.19 -1.48 0.53 0.35 4.14 1 6.48 7.47 ...
 $ vwind : num [1:7300] 1.52 2.022 2.716 1.089 0.675 ...
 $ fapar : num [1:7300] 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.23 0.23 ...
 $ lai   : num [1:7300] 0 0 0 0 0 0 0 0 0.3 0.3 ...
 $ co2   : num [1:7300] 385 378 386 402 437 ...
 $ ccov  : num [1:7300] 0.951 0.951 0.951 0.951 0.951 ...
 $ gpp   : num [1:7300] 0.2188

In [7]:
# check drivers consistency with FluxDataKit
drivers_fdk <- read_rds("data/FLX_CH-Oe2_FLUXNET2015_FULLSET_2004-2023_1-3/rsofun_driver_data_v3.4.2.rds")
drivers_fdk <- drivers_fdk |> filter(sitename == "CH-Oe2")
colnames(drivers_fdk$forcing[[1]])
str(drivers_fdk$forcing[[1]])

[1] "date"   "temp"   "vpd"    "ppfd"   "netrad" "patm"   "snow"   "rain"  
 [9] "tmin"   "tmax"   "vwind"  "fapar"  "co2"    "ccov"   "gpp"    "gpp_qc"
[17] "nee"    "nee_qc" "le"     "le_qc"

tibble [6,205 x 20] (S3: tbl_df/tbl/data.frame)
 $ date  : Date[1:6205], format: "2004-01-01" "2004-01-02" ...
 $ temp  : num [1:6205] -0.212 -0.561 -3.01 -3.709 -1.013 ...
 $ vpd   : num [1:6205] 9.29 27.48 36.61 64.88 33.27 ...
 $ ppfd  : num [1:6205] 1.44e-04 7.21e-05 1.26e-04 6.30e-05 1.01e-04 ...
 $ netrad: logi [1:6205] NA NA NA NA NA NA ...
 $ patm  : num [1:6205] 95774 95492 96239 96384 96726 ...
 $ snow  : num [1:6205] 0 0 0 0 0 0 0 0 0 0 ...
 $ rain  : num [1:6205] 1.85e-05 5.79e-06 0.00 0.00 0.00 ...
 $ tmin  : num [1:6205] -1.33 -2.06 -5.82 -10.45 -8.1 ...
 $ tmax  : num [1:6205] -0.00999 -0.04999 -2.19001 -1.47999 0.52999 ...
 $ vwind : num [1:6205] 1.426 2.022 2.716 1.089 0.675 ...
 $ fapar : num [1:6205] 0.447 0.447 0.448 0.448 0.448 ...
 $ co2   : num [1:6205] 378 378 378 378 378 ...
 $ ccov  : num [1:6205] 0.951 0.938 0.967 0.645 0.891 ...
 $ gpp   : num [1:6205] 0.2079 0.1185 -0.0668 -0.1964 0.0406 ...
 $ gpp_qc: num [1:6205] 0.25 0.3333 0.9583 0.2083 0.0208 ...
 $ ne

In [8]:
# Create directory if it doesn't exist
if (!dir.exists("comparison_plots")) dir.create("comparison_plots")

# Variables to plot
variables <- c('temp', 'vpd', 'ppfd', 'patm', 'snow', 'rain', 
         'tmin', 'tmax', 'vwind', 'fapar', 'co2', 'ccov', 'nee')

# Join datasets
tmp <- drivers_fdk$forcing[[1]] |> 
  right_join(
  drivers$forcing[[1]],
  by = join_by(date),
  suffix = c("_fdk", "_muh")
  )

# Function to create and save scatter plot
create_comparison_plot <- function(var_name, data, index) {
  # Format index for filename (ensure 2 digits)
  idx <- sprintf("%02d", index)
  
  # Create scatter plot
  p1 <- data |> 
  ggplot(aes_string(paste0(var_name, "_fdk"), paste0(var_name, "_muh"))) +
  geom_point(alpha = 0.3) +
  geom_abline(slope = 1, intercept = 0, linetype = "dotted") +
  labs(title = paste("Comparison of", var_name),
     x = paste(var_name, "FDK"),
     y = paste(var_name, "MUH"))
  
  # Create time series plot
  p2 <- data |> 
  ggplot() +
  geom_line(aes(date, !!sym(paste0(var_name, "_muh"))), color = "red") +
  geom_line(aes(date, !!sym(paste0(var_name, "_fdk")))) +
  labs(title = paste("Time series of", var_name),
     x = "Date", 
     y = var_name)
  
  # Combine plots
  combined_plot <- p1 + p2 + plot_layout(ncol = 2)
  
  # Save plot
  ggsave(paste0("comparison_plots/", idx, "_new_", var_name, ".png"), 
     combined_plot, 
     width = 10, height = 5)
  
  return(combined_plot)
}

# Create plots for all variables
plots <- map2(variables, 1:length(variables), 
       ~create_comparison_plot(.x, tmp, .y))

# Print message
cat("All comparison plots saved in 'comparison_plots' directory\n")


Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_point()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_line()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_point()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_line()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_point()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_line()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_point()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the scale range
(`geom_line()`)."
Warning message:
"Removed 1095 rows containing missing values or values outside the 

All comparison plots saved in 'comparison_plots' directory
